# Testfile - to be edited

probably necessary libraries?

In [ ]:
!pip install torch torchvision opencv-python matplotlib

necessary imports?

In [ ]:
import os
import cv2
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim

define dataset clas to read images and labels?

In [ ]:
class ImageClassificationDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.images = sorted([os.path.join(root, file) for file in os.listdir(root) if file.endswith('.jpg')])
        self.labels = sorted([os.path.join(root, file) for file in os.listdir(root) if file.endswith('.txt')])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label_path = self.labels[idx]

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        with open(label_path, 'r') as f:
            label = int(f.read().strip())

        if self.transform:
            img = self.transform(img)

        return img, label

Define transformations and create dataset instance

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

dataset = ImageClassificationDataset(root='path/to/your/data', transform=transform)

Create a data loader

In [ ]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)

Define a simple CNN model

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 56 * 56, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = SimpleCNN(num_classes=10)  # Set the number of classes


Define the loss function, optimizer, and learning rate scheduler

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

Train the model

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    scheduler.step()

Evaluate the model on test data (use a different dataset instance for test data)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in test_dataloader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        outputs = model(imgs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum